In [ ]:
import os
import shutil
import subprocess
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
import re
os.chdir("D:/uppsala/15. ds projects/data")

# #  Tesseract 
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# ============================================================
# 🧩 1： PDF  PDF
# ============================================================

def convert_to_single_column(pdf_path, output_pdf, k2_path=None, timeout=600):
    """
     k2pdfopt  PDF ，。
    """
    exe = k2_path or shutil.which("k2pdfopt") or shutil.which("k2pdfopt.exe")
    if not exe or (k2_path and not os.path.exists(k2_path)):
        raise FileNotFoundError(f" k2pdfopt， k2_path  PATH。 k2_path={k2_path}")

    cmd = [
        exe, pdf_path,
        "-o", output_pdf,
        "-mode", "copy",      #  reflow ，
        "-col", "2",        #  Note.        "-dev", "pdf",      #   PDF
        "-ui-",             #  Note.        "-p", "1-9999",     #  Note.        "-x",               #  Note.        "-wrap-",           #  Note.        "-n",               #  Note.        "-autorotate-",     #  ✅ 
        "-rt", "0"          #  ✅  0°
    ]

    res = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
    if res.returncode != 0:
        raise RuntimeError(f"k2pdfopt ：{res.stderr or res.stdout}")

    if not os.path.exists(output_pdf) or os.path.getsize(output_pdf) < 5000:
        raise RuntimeError(f"⚠️ ，。")

    print(f"✅ ：{output_pdf} （ {os.path.getsize(output_pdf)//1024} KB）")
    return output_pdf




# ============================================================
# 🧩 2： PDF （PyMuPDF）
# ============================================================
def extract_text_direct(pdf_path, output_txt="direct_text.txt"):
    """
     PyMuPDF  PDF 。
    """
    doc = fitz.open(pdf_path)
    all_text = []
    print("【1: PyMuPDF 】")
    for i, page in enumerate(doc, start=1):
        text = page.get_text("text")
        print(f"→  {i}  {len(text)} ")
        print("=" * 60)
        print(text[:500])
        print("=" * 60)
        all_text.append(text)
    doc.close()

    full_text = "\n".join(all_text)
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(full_text)
    print(f"✅ PyMuPDF ：{output_txt}")
    return full_text


# ============================================================
# 🧩 3：OCR  PDF （Tesseract）
# ============================================================
def extract_text_ocr(pdf_path, output_txt="ocr_text.txt", lang="eng"):
    """
     Tesseract OCR  PDF 。
    """
    doc = fitz.open(pdf_path)
    all_text = []
    print("【2: OCR 】")
    for i, page in enumerate(doc, start=1):
        print(f"→  {i}  ...")
        pix = page.get_pixmap(dpi=300)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img = img.convert("L")
        ocr_text = pytesseract.image_to_string(img, lang=lang)
        print("=" * 60)
        print(ocr_text[:400])
        print("=" * 60)
        all_text.append(ocr_text)
    doc.close()

    full_text = "\n".join(all_text)
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(full_text)
    print(f"✅ OCR ：{output_txt}")
    return full_text

def process_pdf(mode, pdf_path, k2_path):
    """
    mode = 1 :  + 
    mode = 2 :  + OCR 
    """
    # Note.    base_name = os.path.splitext(os.path.basename(pdf_path))[0]  #  （）
    single_column_pdf = f"{base_name}_single_column.pdf"  #   PDF 
    ocr_output_txt = f"{base_name}_ocr_txt.txt"  #  OCR 

    print(f"🚀 Step1: ...")
    convert_to_single_column(pdf_path, single_column_pdf, k2_path=k2_path)

    if mode == 2:
        print(f"🚀 Step2: OCR （Tesseract）")
        extract_text_ocr(single_column_pdf, ocr_output_txt)
    else:
        raise ValueError("mode  2")

    print(f"✅ ！OCR ：{ocr_output_txt}")



In [ ]:

# ============================================================
# ✅ 
# ============================================================
if __name__ == "__main__":
    process_pdf(
        mode=2,   #  2 = OCR
        pdf_path="A_PV-289-EN.pdf",
        k2_path=r"D:\LeStoreDownload\k2pdfopt.exe"
    )


🚀 Step1: 双栏转单栏处理中...
✅ 转换成功：A_PV-289-EN_single_column.pdf （大小 2104 KB）
🚀 Step2: OCR 识别文本（Tesseract）
【方法2: OCR 提取原始文本】
→ 正在识别第 1 页 ...
United Naitons.

GENERAL
ASSEMBLY

FIFTH SESSION
Official Records


→ 正在识别第 2 页 ...
Zooth
PLENARY MEETING

Thursday, 28 September 1950, at 3 p.m,

Flushing Meadow, New York

→ 正在识别第 3 页 ...
CONTENTS

General debate (concluded) ......ccccccccnccccnccccuceceseeeteeserenes 173
Speeches by Mr. Aklilou (Ethiopia) and the Secretary-General
Admission of the-Republic of Indonesia to membership of the United Nations. 176

President: Mr. Nasrollah Entezam (Iran).

→ 正在识别第 4 页 ...
General debate (concluded)
[Agenda item 9]

SPEECHES BY Mr. Ax titou (ETHIOPIA) AND THE
SECRETARY-GENERAL

1. Mr. AKLILOU (Ethiopia) (translated from
French): A year ago, when the fourth session of the
General Assembly met, there was an atmosphere of
optimism and determination to accomplish the work
before us. The President of that session, General
Romulo, expressed the hope that the four